# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
data_qty = data.groupby(["CustomerID","ProductName"]).agg({"Quantity":"sum"})
data_qty.head()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
data_pivot = pd.pivot_table(data_qty, values="Quantity", index="ProductName", columns="CustomerID", fill_value=0)
data_pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
simil_matrix = pd.DataFrame(1/(1 + squareform(pdist(data_pivot.T, 'euclidean'))),index=data_pivot.columns, columns=data_pivot.columns)
simil_matrix.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [7]:
def topSimilar(ID):
    return list(simil_matrix[ID].sort_values(ascending=False)[1:6].index)

In [8]:
lista_top = topSimilar(33)
lista_top

[264, 3535, 3317, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [9]:
data_qtyTOP = data_qty.loc[[e for e in lista_top]]
data_qtyTOP

Quantity
CustomerID ProductName                             
264        Apricots - Halves                      1
           Apricots Fresh                         1
           Bacardi Breezer - Tropical             1
           Bagel - Plain                          1
           Banana - Leaves                        1
...                                             ...
3535       Tray - 16in Rnd Blk                    1
           Wanton Wrap                            1
           Wine - Blue Nun Qualitatswein          1
           Wine - Magnotta, Merlot Sr Vqa         1
           Yoghurt Tubes                          1

[270 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [10]:
data_qtyTOP = data_qtyTOP.groupby("ProductName").agg({"Quantity":"sum"}).sort_values("Quantity",ascending=False)
data_qtyTOP 

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3
...,...
Hinge W Undercut,1
Ice Cream Bar - Hageen Daz To,1
Jagermeister,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [11]:
#Hago un merge del df de los datos del customer33 con los productos de los 5 clientes más similares
merge_ID33 = (pd.DataFrame(data_pivot[33])).merge(right=data_qtyTOP, how='left', on="ProductName")
merge_ID33.head()

,33,Quantity
ProductName,,
Anchovy Paste - 56 G Tube,0,1.0
"Appetizer - Mini Egg Roll, Shrimp",0,NaN
Appetizer - Mushroom Tart,0,1.0
Appetizer - Sausage Rolls,0,1.0
Apricots - Dried,1,1.0


In [12]:
# Me quedo solo con los productos que el customer 33 aun no ha comprado. Y ordeno por Quantity
merge_ID33 = merge_ID33.drop(index=(merge_ID33[merge_ID33[33]!=0].index)).sort_values("Quantity",ascending=False)
merge_ID33

,33,Quantity
ProductName,,
Wine - Blue Nun Qualitatswein,0,3.0
Soup - Campbells Bean Medley,0,3.0
Wine - Ej Gallo Sierra Valley,0,3.0
Butter - Unsalted,0,3.0
Yoghurt Tubes,0,2.0
...,...,...
"Wine - White, Colubia Cresh",0,NaN
"Wine - White, Mosel Gold",0,NaN
Yeast Dry - Fermipan,0,NaN


In [13]:
#Saco la lista de los 5 primeros índices
print(list(merge_ID33.index[:5]))

['Wine - Blue Nun Qualitatswein', 'Soup - Campbells Bean Medley', 'Wine - Ej Gallo Sierra Valley', 'Butter - Unsalted', 'Yoghurt Tubes']


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [14]:
recomm = {}
CustomersIDs = [e for e in data_pivot.columns]
for e in CustomersIDs:
    lista_top = topSimilar(e) #Funcion que me da una lista con el top 5 de customers más similares al customer elegido
    data_qtyTOP = data_qty.loc[[customer for customer in lista_top]]
    data_qtyTOP = data_qtyTOP.groupby("ProductName").agg({"Quantity":"sum"}).sort_values("Quantity",ascending=False)
    merge_ID = (pd.DataFrame(data_pivot[e])).merge(right=data_qtyTOP, how='left', on="ProductName") 
    merge_ID = merge_ID.drop(index=(merge_ID[merge_ID[e]!=0].index)).sort_values("Quantity",ascending=False)
    recomm[e] = list(merge_ID.index[:5])

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [25]:
recommendations = pd.DataFrame(recomm.values(),index=recomm.keys())
recommendations

,0,1,2,3,4
33,Wine - Blue Nun Qualitatswein,Soup - Campbells Bean Medley,Wine - Ej Gallo Sierra Valley,Butter - Unsalted,Yoghurt Tubes
200,Soup - Campbells Bean Medley,Muffin - Carrot Individual Wrap,Wanton Wrap,Bay Leaf,Pork - Kidney
264,Soupfoamcont12oz 112con,Wine - Two Oceans Cabernet,"Veal - Inside, Choice",Bread - Italian Roll With Herbs,Fondant - Icing
356,"Veal - Inside, Choice",Lamb - Ground,Butter - Unsalted,Sardines,Beef - Inside Round
412,"Sauce - Gravy, Au Jus, Mix",Sprouts - Baby Pea Tendrils,Olive - Spread Tapenade,Wine - Blue Nun Qualitatswein,Cheese - Cambozola
...,...,...,...,...,...
97928,"Soup - Campbells, Lentil","Cheese - Brie,danish",Pants Custom Dry Clean,Wiberg Super Cure,Shrimp - 31/40
98069,Skirt - 29 Foot,Milk - 1%,Cheese - Taleggio D.o.p.,Beans - Kidney White,Sprouts - Baby Pea Tendrils
98159,"Lamb - Whole, Fresh",Tea - English Breakfast,Chips Potato Salt Vinegar 43g,Bread - Raisin Walnut Oval,Wine - Redchard Merritt
98185,Crackers - Trio,Pernod,Tea - Jasmin Green,"Peas - Pigeon, Dry",Pastry - Choclate Baked


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

Types of distance metric: ‘braycurtis’, ‘canberra’, ‘chebyshev’, ‘cityblock’, ‘correlation’, ‘cosine’, ‘dice’, ‘euclidean’, ‘hamming’, ‘jaccard’, ‘jensenshannon’, ‘kulsinski’, ‘mahalanobis’, ‘matching’, ‘minkowski’, ‘rogerstanimoto’, ‘russellrao’, ‘seuclidean’, ‘sokalmichener’, ‘sokalsneath’, ‘sqeuclidean’, ‘yule’.

In [29]:
simil_matrix = pd.DataFrame(1/(1 + squareform(pdist(data_pivot.T, 'correlation'))),index=data_pivot.columns, columns=data_pivot.columns)
display(simil_matrix.head())
recomm = {}
CustomersIDs = [e for e in data_pivot.columns]
for e in CustomersIDs:
    lista_top = topSimilar(e) #Funcion que me da una lista con el top 5 de customers más similares al customer elegido
    data_qtyTOP = data_qty.loc[[customer for customer in lista_top]]
    data_qtyTOP = data_qtyTOP.groupby("ProductName").agg({"Quantity":"sum"}).sort_values("Quantity",ascending=False)
    merge_ID = (pd.DataFrame(data_pivot[e])).merge(right=data_qtyTOP, how='left', on="ProductName") 
    merge_ID = merge_ID.drop(index=(merge_ID[merge_ID[e]!=0].index)).sort_values("Quantity",ascending=False)
    recomm[e] = list(merge_ID.index[:5])
recommendations = pd.DataFrame(recomm.values(),index=recomm.keys())
recommendations

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.493776,0.529677,0.510360,0.506951,0.507217,0.487410,0.497932,0.499215,0.502499,...,0.493385,0.484376,0.493486,0.479867,0.499870,0.479764,0.493197,0.486457,0.491384,0.492072
200,0.493776,1.000000,0.499733,0.498575,0.482172,0.481921,0.519139,0.480599,0.499131,0.505459,...,0.504719,0.496170,0.517884,0.508954,0.492581,0.484514,0.534491,0.500253,0.484272,0.488735
264,0.529677,0.499733,1.000000,0.497579,0.513573,0.490305,0.503243,0.480936,0.502028,0.490242,...,0.508803,0.490805,0.499750,0.499162,0.517743,0.525278,0.511845,0.503500,0.515736,0.490875
356,0.510360,0.498575,0.497579,1.000000,0.490143,0.489778,0.501933,0.507803,0.482686,0.486354,...,0.499105,0.486660,0.498662,0.502228,0.494239,0.496086,0.484207,0.502204,0.506401,0.500631
412,0.506951,0.482172,0.513573,0.490143,1.000000,0.497577,0.505111,0.512470,0.493947,0.507777,...,0.488109,0.529502,0.492033,0.495264,0.504752,0.500064,0.481259,0.502418,0.505997,0.515015


,0,1,2,3,4
33,Knife Plastic - White,Muffin - Zero Transfat,Banana Turning,Veal - Osso Bucco,Crush - Cream Soda
200,Otomegusa Dashi Konbu,Potatoes - Idaho 100 Count,Crackers - Trio,Milk Powder,Pail With Metal Handle 16l White
264,"Water - Mineral, Natural",Wine - Toasted Head,Snapple - Iced Tea Peach,Hersey Shakes,Garbag Bags - Black
356,Cheese - Taleggio D.o.p.,"Coconut - Shredded, Sweet",Ocean Spray - Kiwi Strawberry,Cheese - Cheddarsliced,Soup Knorr Chili With Beans
412,Butter - Unsalted,Cake - Mini Cheesecake,"Salmon - Atlantic, Skin On",Wine - Hardys Bankside Shiraz,Gloves - Goldtouch Disposable
...,...,...,...,...,...
97928,Lettuce - Treviso,"Yogurt - Blueberry, 175 Gr",Bread - Calabrese Baguette,Extract - Lemon,Coffee - Irish Cream
98069,"Wine - Red, Colio Cabernet",Veal - Inside,Soupfoamcont12oz 112con,Sprouts - Baby Pea Tendrils,Peas - Frozen
98159,"Water, Tap",Bananas,Wine - Redchard Merritt,"Fish - Scallops, Cold Smoked",Wine - Wyndham Estate Bin 777
98185,Squid U5 - Thailand,Cheese - Taleggio D.o.p.,"Peas - Pigeon, Dry",Salmon Steak - Cohoe 8 Oz,"Sole - Dover, Whole, Fresh"


In [30]:
simil_matrix = pd.DataFrame(1/(1 + squareform(pdist(data_pivot.T, 'cityblock'))),index=data_pivot.columns, columns=data_pivot.columns)
display(simil_matrix.head())
recomm = {}
CustomersIDs = [e for e in data_pivot.columns]
for e in CustomersIDs:
    lista_top = topSimilar(e) #Funcion que me da una lista con el top 5 de customers más similares al customer elegido
    data_qtyTOP = data_qty.loc[[customer for customer in lista_top]]
    data_qtyTOP = data_qtyTOP.groupby("ProductName").agg({"Quantity":"sum"}).sort_values("Quantity",ascending=False)
    merge_ID = (pd.DataFrame(data_pivot[e])).merge(right=data_qtyTOP, how='left', on="ProductName") 
    merge_ID = merge_ID.drop(index=(merge_ID[merge_ID[e]!=0].index)).sort_values("Quantity",ascending=False)
    recomm[e] = list(merge_ID.index[:5])
recommendations = pd.DataFrame(recomm.values(),index=recomm.keys())
recommendations

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.008000,0.009901,0.009174,0.008403,0.008929,0.007752,0.008850,0.008621,0.008696,...,0.000564,0.000596,0.000556,0.000674,0.000542,0.000569,0.000556,0.000554,0.000476,0.000534
200,0.008000,1.000000,0.008130,0.007874,0.006993,0.007576,0.007752,0.007752,0.008065,0.008130,...,0.000563,0.000593,0.000557,0.000675,0.000538,0.000567,0.000558,0.000553,0.000473,0.000531
264,0.009901,0.008130,1.000000,0.008547,0.008850,0.008475,0.007874,0.008403,0.008621,0.008264,...,0.000566,0.000596,0.000556,0.000678,0.000543,0.000576,0.000559,0.000557,0.000479,0.000533
356,0.009174,0.007874,0.008547,1.000000,0.008000,0.008197,0.007752,0.009174,0.008065,0.008130,...,0.000564,0.000595,0.000556,0.000676,0.000540,0.000571,0.000553,0.000556,0.000478,0.000534
412,0.008403,0.006993,0.008850,0.008000,1.000000,0.008065,0.007752,0.008850,0.007937,0.008264,...,0.000561,0.000599,0.000553,0.000674,0.000540,0.000570,0.000552,0.000555,0.000477,0.000536


,0,1,2,3,4
33,"Oranges - Navel, 72",Chocolate - Dark,Pecan Raisin - Tarts,Bay Leaf,Wine - Charddonnay Errazuriz
200,Soup - Campbells Bean Medley,Bay Leaf,Pork - Kidney,Scampi Tail,Muffin - Carrot Individual Wrap
264,"Veal - Inside, Choice",Bread - Italian Roll With Herbs,Muffin - Carrot Individual Wrap,Wine - Two Oceans Cabernet,Black Currants
356,Butter - Unsalted,"Veal - Inside, Choice","Nut - Chestnuts, Whole","Beets - Candy Cane, Organic",Raspberries - Fresh
412,Wine - Blue Nun Qualitatswein,Soup - Campbells Bean Medley,Table Cloth 81x81 White,"Pepper - Black, Whole",Butter - Unsalted
...,...,...,...,...,...
97928,Wine - Chardonnay South,Butter - Unsalted,Raspberries - Fresh,Cream Of Tartar,Sobe - Tropical Energy
98069,Wine - Chardonnay South,Pork - Hock And Feet Attached,Bar - Granola Trail Mix Fruit Nut,"Mushrooms - Black, Dried",Extract - Lemon
98159,Sherry - Dry,Bandage - Fexible 1x3,"Wine - Magnotta, Merlot Sr Vqa","Pepper - Black, Whole",Knife Plastic - White
98185,Muffin - Carrot Individual Wrap,Beef - Montreal Smoked Brisket,Pork - Kidney,Berry Brulee,Tia Maria
